# Tutorial on Tuning Hyperparameters of Retrieval Systems

This tutorial exemplifies hyperparameter tuning with PyTerrier and TrecTools.
To make things a bit more explicit, we exhaustively evaluate a small grid of possible parameters for BM25.
After you understand the concepts of this tutorial, please consider to switch to a dedicated API for tuning hyperparameters, e.g., [the official PyTerrier one](https://pyterrier.readthedocs.io/en/latest/tuning.html).

**Attention:** This tutorial comes in two parts, where part 1 executes all configurations and part 2 does the actual search. Please skim only over part 1 (and do not execute it) if you do this tutorial for the first time and come back later if needed, as we prepared the results of part 2 via a download so that you directly can skip to part 2.

## Preparation: Install dependencies

In [1]:
# This is only needed in Google Colab, in the dev container, everything should be installed already
!pip3 install tira trectools python-terrier

## Our Scenario

We want to tune the hyperparameters of BM25 on the training and validation data of the [IR Lab of the winter semester Jena/Leipzig](https://www.tira.io/task-overview/ir-lab-jena-leipzig-wise-2023).

First, we import all used dependencies:



In [2]:
from tira.third_party_integrations import ir_datasets, ensure_pyterrier_is_loaded, persist_and_normalize_run
import pyterrier as pt

ensure_pyterrier_is_loaded()

training_dataset = 'ir-lab-jena-leipzig-wise-2023/training-20231104-training'
validation_dataset = 'ir-lab-jena-leipzig-wise-2023/validation-20231104-training'

Start PyTerrier with version=5.7, helper_version=0.0.7, no_download=True


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


# Part 1: Run all Configurations of the Grid Search

Running all configurations below takes roughly three hours (that is one advantage of dedicated APIs, they often offer parallelization). Therefore, please only skim over this first part if you do the tutorial for the first time, you can download the outputs of this grid search at the beginning of part 2 so that you directly can skip to part 2.

Next, we implement two methods: (1) for indexing documents, and (2) for grid search method to exhaustively run a small grid of possible parameters for BM25. We will store all runs in a directory `grid-search/training` (for the training runs) respectively `grid-search/validation` (for the validation runs).

In [3]:
def create_index(documents):
    indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480})
    index_ref = indexer.index(({'docno': i.doc_id, 'text': i.text} for i in documents))
    return pt.IndexFactory.of(index_ref)

In [4]:
def run_bm25_grid_search_run(index, output_dir, queries):
    """
        defaults: http://terrier.org/docs/current/javadoc/org/terrier/matching/models/BM25.html
        k_1 = 1.2d, k_3 = 8d, b = 0.75d
        We do not tune parameter k_3, as this parameter only impacts queries with reduntant terms.
    """
    for b in [0.7, 0.75, 0.8]:
        for k_1 in [1.1, 1.2, 1.3]:
            system = f'bm25-b={b}-k_1={k_1}'
            configuration = {"bm25.b" : b, "bm25.k_1": k_1}
            run_output_dir = output_dir + '/' + system
            !rm -Rf {run_output_dir}
            !mkdir -p {run_output_dir}
            print(f'Run {system}')
            BM25 = pt.BatchRetrieve(index, wmodel="BM25", controls=configuration, verbose=True)
            run = BM25(queries)
            persist_and_normalize_run(run, system, run_output_dir)

## Run All Configurations on the Training Data

First, we load the training dataset and index the documents, then we run our `run_bm25_grid_search_run`.

In [5]:
dataset = ir_datasets.load(training_dataset)
queries = pt.io.read_topics(ir_datasets.topics_file(training_dataset), format='trecxml')

queries.head(3)

Load ir_dataset "ir-lab-jena-leipzig-wise-2023/training-20231104-training" from tira.
No settings given in /root/.tira/.tira-settings.json. I will use defaults.


,qid,query
0,q06223196,car shelter
1,q062228,airport
2,q062287,antivirus comparison


In [6]:
index = create_index(dataset.docs_iter())

No settings given in /root/.tira/.tira-settings.json. I will use defaults.


In [7]:
run_bm25_grid_search_run(index, 'grid-search/training', queries)

Run bm25-b=0.7-k_1=1.1


BR(BM25): 100%|██████████| 672/672 [07:49<00:00,  1.43q/s]


Done. run file is stored under "grid-search/training/bm25-b=0.7-k_1=1.1/run.txt".
Run bm25-b=0.7-k_1=1.2


BR(BM25): 100%|██████████| 672/672 [08:39<00:00,  1.29q/s]


Done. run file is stored under "grid-search/training/bm25-b=0.7-k_1=1.2/run.txt".
Run bm25-b=0.7-k_1=1.3


...

Done. run file is stored under "grid-search/training/bm25-b=0.8-k_1=1.1/run.txt".
Run bm25-b=0.8-k_1=1.2


BR(BM25): 100%|██████████| 672/672 [08:34<00:00,  1.31q/s]


Done. run file is stored under "grid-search/training/bm25-b=0.8-k_1=1.2/run.txt".
Run bm25-b=0.8-k_1=1.3


BR(BM25): 100%|██████████| 672/672 [07:48<00:00,  1.44q/s]


Done. run file is stored under "grid-search/training/bm25-b=0.8-k_1=1.3/run.txt".


## Run All Configurations on the Validation Data

Second, we load the validation dataset and index the documents, then we run our `run_bm25_grid_search_run`.

In [ ]:
dataset = ir_datasets.load(validation_dataset)
queries = pt.io.read_topics(ir_datasets.topics_file(validation_dataset), format='trecxml')

queries.head(3)

In [ ]:
index = create_index(dataset.docs_iter())

# Part 2: Evaluate all Configurations of the Grid Search

In [ ]:
from tira.rest_api_client import Client
tira = Client()

tira.download_dataset('ir-lab-jena-leipzig-wise-2023', 'training-20231104-training', truth_dataset=True)

'/root/.tira/extracted_datasets/ir-lab-jena-leipzig-wise-2023/training-20231104-training/truth-data'

In [ ]:
!ls /root/.tira/extracted_datasets/ir-lab-jena-leipzig-wise-2023/training-20231104-training/truth-data

metadata.json  qrels.txt  queries.jsonl  queries.xml


# Summary

We ...

To summarize everything, please answer the following three questions:


### Question 1:

What are the advantages of splitting  into a training and validation set?


### Question 2:

Are there scenarious where you would join the training and validation data?